# Importing libraries

In [ ]:

import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
#from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, GlobalMaxPooling1D,Dense, Dropout, Flatten, MaxPooling1D, Convolution2D, MaxPooling2D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import csv
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import shuffle
import keras
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)


from matplotlib import pyplot
from numpy import where
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn import metrics

# Loading Images

In [45]:
def grab_image(img_path):
    img = image.load_img(img_path, target_size=(227, 227))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x
num_imgs = 800 # number of images to use
X = np.concatenate([grab_image('%06d.jpg'%(i)) for i in range(1,num_imgs+1)])
print("X.shape: ",X.shape)

X.shape:  (800, 227, 227, 3)


# Spliting the original data into train and test data

In [46]:
y = pd.read_csv('list_attr_celeba.csv', index_col=0)
Y = y["Male"].iloc[:num_imgs]
print("leny: ",len(Y))
Y = Y.apply(lambda x: 0 if x==-1 else x)
#Y = keras.utils.to_categorical(Y ,num_classes=2)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

print("X_train.shape: ",X_train.shape)

leny:  800
X_train.shape:  (640, 227, 227, 3)


# Creating the AlexNet Model

In [47]:
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4)))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid"))
model.add(Activation("relu"))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))


# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(227*227*3,)))
model.add(Activation("relu"))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation("relu"))
# Add Dropout
model.add(Dropout(0.4))
# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation("relu"))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.summary()
#opt = keras.optimizers.Adam(learning_rate=0.0001)
# Compile the model
model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=["accuracy"]) 


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_63 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 23, 23, 256)       614656    
_________________________________________________________________
activation_64 (Activation)   (None, 23, 23, 256)       0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 9, 9, 384)        

# Running the model

In [48]:
batch_size = 32
epochs = 10
history=model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs, batch_size=batch_size)


Epoch 1/10
20/20 [==============================] - 38s 2s/step - loss: 1.0536 - accuracy: 0.5172 - val_loss: 0.6906 - val_accuracy: 0.5437
Epoch 2/10
20/20 [==============================] - 36s 2s/step - loss: 0.6843 - accuracy: 0.5719 - val_loss: 0.6844 - val_accuracy: 0.5437
Epoch 3/10
20/20 [==============================] - 36s 2s/step - loss: 0.6717 - accuracy: 0.5719 - val_loss: 0.6672 - val_accuracy: 0.5500
Epoch 4/10
20/20 [==============================] - 36s 2s/step - loss: 0.6214 - accuracy: 0.6313 - val_loss: 0.6642 - val_accuracy: 0.6187
Epoch 5/10
20/20 [==============================] - 36s 2s/step - loss: 0.5700 - accuracy: 0.7156 - val_loss: 0.5500 - val_accuracy: 0.7063
Epoch 6/10
20/20 [==============================] - 36s 2s/step - loss: 0.4880 - accuracy: 0.7625 - val_loss: 0.4884 - val_accuracy: 0.7625
Epoch 7/10
20/20 [==============================] - 36s 2s/step - loss: 0.4154 - accuracy: 0.7937 - val_loss: 0.3271 - val_accuracy: 0.8687
Epoch 8/10
20/20 [==

# Accuracy of the model to detect a person is male or not.

In [53]:
scores = model.evaluate((X_test),y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 91.25%
